

### 6.4210 Final Project
Some basic examples are from the [textbook](http://manipulation.csail.mit.edu/intro.html).  I recommend to find some important information in the textbook.

In this project, it is intended to be a multi-step process that will culminate in a simulated demonstration of these robots throwing and catching a simple object back and forth repeatedly in which the catching robot will have to move. If time permits we will attempt to replicate at least one successful throw and catch on real-world hardware.

In [1]:
# import some inportant libaray
# import os
# import sys
# import warnings
from enum import Enum
import numpy as np
from manipulation.utils import running_as_notebook
from copy import deepcopy
import pydot
import matplotlib.pyplot as plt
from IPython.display import HTML, SVG, display
from pydrake.examples.manipulation_station import ManipulationStation

import pydrake

from scenarios import *

from pydrake.all import (ConstantVectorSource, DiagramBuilder,
                         FindResourceOrThrow, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant, Parser,
                         PiecewisePolynomial, PiecewiseQuaternionSlerp,
                         RigidTransform, RollPitchYaw, PointCloud, LeafSystem, RotationMatrix,
                         PiecewisePose, Simulator, Solve, StartMeshcat, TrajectorySource,
                         Concatenate, BasicVector)

from pydrake.multibody import inverse_kinematics

# from pydrake.all import (
#     AbstractValue, Adder, AddMultibodyPlantSceneGraph, BallRpyJoint, BaseField,
#     Box, CameraInfo, ClippingRange, CoulombFriction, Cylinder, Demultiplexer,
#     DepthImageToPointCloud, DepthRange, DepthRenderCamera, DiagramBuilder,
#     FindResourceOrThrow, GeometryInstance, InverseDynamicsController,
#     LeafSystem, LoadModelDirectivesFromString,
#     MakeMultibodyStateToWsgStateSystem, MakePhongIllustrationProperties,
#     MakeRenderEngineVtk, ModelInstanceIndex, MultibodyPlant, Parser,
#     PassThrough, PrismaticJoint, ProcessModelDirectives, RenderCameraCore,
#     RenderEngineVtkParams, RevoluteJoint, Rgba, RgbdSensor, RigidTransform,
#     RollPitchYaw, RotationMatrix, SchunkWsgPositionController, SpatialInertia,
#     Sphere, StateInterpolatorWithDiscreteDerivative, UnitInertia)

# from pydrake.manipulation.planner import (
#     DifferentialInverseKinematicsIntegrator,
#     DifferentialInverseKinematicsParameters)

# from manipulation.utils import AddPackagePaths, FindResource 

# not quite sure whether we need these,but I just modify the code from the example code in the textbook
from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton

from manipulation.scenarios import (AddIiwaDifferentialIK,
                                    MakeManipulationStation)

from manipulation.meshcat_utils import AddMeshcatTriad
from grasp_pose_planner import *
from throwing_calculations import *

from manipulation.scenarios import AddMultibodyTriad
from manipulation.clutter import GenerateAntipodalGraspCandidate
from pydrake.trajectories import PiecewisePolynomial

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://1e147b87-4611-40c9-8663-f05c09154240.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


### Load the model

This is the block that contains: 2 iiwa robots, 2 bins and 6 cameras.
The physics and geometry engines running in the simulation above are actually running in 3D.  This example is almost identical, but we'll use the (default) 3D visualization and add more sliders for controlling the full `roll`, `pitch`, `yaw` angles and `x`, `y`, `z` positions of the end effector.

In [3]:
# how much distance we want between the robots
x_distance = -1.5
y_distance = 1.5
robot1_transform = RigidTransform([0.0,0,0])
assert((x_distance**2 + y_distance**2)**0.5 >= 1.0)

model_directives = """
directives:
# add the iiwa robot 1: iiwa(throwing robot)

- add_frame:
    name: iiwa_frame
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0.0, 0.0 ]}
      translation: [0.0, 0, 0.0]

- add_model:

    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: iiwa_frame
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

- add_frame:
    name: iiwa2_frame
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0.0, 90.0 ]}
      translation: ["""+str(x_distance)+""","""+str(y_distance)+""" , -0.015]

# add the iiwa robot 2: iiwa(catching robot)
- add_model:
    name: iiwa2
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: iiwa2_frame
    child: iiwa2::iiwa_link_0
- add_model:
    name: wsg1
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa2::iiwa_link_7
    child: wsg1::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}

# add the bin model
# bin 0 (near the throwing robot)
- add_frame:
    name: bin0_origin
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0.0, 90.0 ]}
      translation: [-0.05, -0.5, -0.015]
- add_model:
    name: bin0
    file: package://drake/examples/manipulation_station/models/bin.sdf
- add_weld:
    parent: bin0_origin
    child: bin0::bin_base

# bin 1 (near the catching robot)
- add_frame:
    name: bin1_origin
    X_PF:
      base_frame: world
      rotation: !Rpy { deg: [0.0, 0.0, 180.0 ]}
      translation: ["""+str((x_distance+.5))+""","""+str((y_distance+0.2))+""" , -0.015]
- add_model:
    name: bin1
    file: package://drake/examples/manipulation_station/models/bin.sdf
- add_weld:
    parent: bin1_origin
    child: bin1::bin_base

- add_model:
    name: floor
    file: package://manipulation/floor.sdf

- add_weld:
    parent: world
    child: floor::box
    X_PC:
        translation: [0, 0, -.5]

- add_frame:
    name: camera0_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [-130.0, 0, 90.0]}
        translation: [.25, -.5, .4]

- add_model:
    name: camera0
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera0_origin
    child: camera0::base

- add_frame:
    name: camera1_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [-150., 0, 0.0]}
        translation: [-0.05, -.7, .5]

- add_model:
    name: camera1
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera1_origin
    child: camera1::base

- add_frame:
    name: camera2_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: ["""+str(-140.0)+""", 0, -120.0]}
        translation: [-.35, -.25, .45]

- add_model:
    name: camera2
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera2_origin
    child: camera2::base

- add_frame:
    name: camera3_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [-130.0, 0, 0.0]}
        translation: ["""+str(x_distance+.6)+""", """+str(y_distance+0.2-.3)+""", .4]

- add_model:
    name: camera3
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera3_origin
    child: camera3::base

- add_frame:
    name: camera4_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [-150., 0, """+str(-90)+"""]}
        translation: ["""+str(x_distance+.3)+""", """+str(y_distance+0.2+.05)+""", .5]

- add_model:
    name: camera4
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera4_origin
    child: camera4::base

- add_frame:
    name: camera5_origin
    X_PF:
        base_frame: world
        rotation: !Rpy { deg: [-140., 0, """+str(-150.0-90.0)+"""]}
        translation: ["""+str(x_distance+.75)+""", """+str(y_distance+0.2+.35)+""", .45]

- add_model:
    name: camera5
    file: package://manipulation/camera_box.sdf

- add_weld:
    parent: camera5_origin
    child: camera5::base

# add foam_brick (the object that robot need to catch)
- add_model:
    name: foam_brick 
    file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf 
    default_free_body_pose:
        base_link:
            translation: ["""+str((x_distance+.5))+""", """+str((y_distance+0.2-.0)-(0.5)*0)+""", 0.2]
- add_model:
    name: foam_brick2 
    file: package://drake/examples/manipulation_station/models/061_foam_brick.sdf 
    default_free_body_pose:
        base_link:
            translation: ["""+str(0)+""", """+str(-(0.62)*1)+""", 0.2]
    
            
"""

# ppackage://drake/examples/manipulation_station/models/cylinder.sdf

### Initialize the two robots and other basic objects
We have two robots
we'll use the (default) 3D visualization and add more sliders for controlling the full `roll`, `pitch`, `yaw` angles and `x`, `y`, `z` positions of the end effector.

In [4]:
    meshcat.Delete()
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    time_step = 0.001
    station = builder.AddSystem(
        MakeManipulationStation(model_directives, time_step=time_step))
    plant.Finalize()
    station_plant = station.GetSubsystemByName("plant")

    controller_plant = station.GetSubsystemByName(
        "iiwa_controller").get_multibody_plant_for_control()


    controller_plant2 = station.GetSubsystemByName(
        "iiwa2_controller").get_multibody_plant_for_control()

    
    gripperOffset = [0.0,0,-0.128]


    # Add a meshcat visualizer.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()

    # For inverse dynamic, disable the default controller

    # default controller start

    # # Set up differential inverse kinematics.
    # differential_ik = AddIiwaDifferentialIK(
    #     builder,
    #     controller_plant,
    #     frame=controller_plant.GetFrameByName("iiwa_link_7"))
    # builder.Connect(differential_ik.get_output_port(),
    #                 station.GetInputPort("iiwa_position"))
    # builder.Connect(station.GetOutputPort("iiwa_state_estimated"),
    #                 differential_ik.GetInputPort("robot_state"))

    # default controller end

    # also disable the position controller by feeding the current position as desired position

    builder.Connect(station.GetOutputPort('iiwa_position_measured'), 
                station.GetInputPort('iiwa_position'))

    builder.Connect(station.GetOutputPort('iiwa2_position_measured'), 
                station.GetInputPort('iiwa2_position'))
 
    # comment out the line above if you want to enable the controller

    # second iiwa stuff
    
    '''
    iiwa2_instance = None
    for i in range(controller_plant2.num_model_instances()):
        model_instance = ModelInstanceIndex(i)
        model_instance_name = controller_plant.GetModelInstanceName(model_instance)

        if model_instance_name.startswith('iiwa'):
            iiwa2_instance = model_instance
            break

    controller_plant2 = station.GetSubsystemByName(
        "iiwa2_controller").get_multibody_plant_for_control()


    assert(iiwa2_instance != None)

    # Set up differential inverse kinematics.
    differential_ik2 = AddIiwaDifferentialIK(
        builder,
        controller_plant2,
        frame=controller_plant2.GetFrameByName("iiwa_link_7"))
    builder.Connect(differential_ik2.get_output_port(),
                    station.GetInputPort("iiwa2_position"))
    builder.Connect(station.GetOutputPort("iiwa2_state_estimated"),
                    differential_ik2.GetInputPort("robot_state"))

    # Set up teleop widgets.
    
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            min_range=MeshcatPoseSliders.MinRange(roll=0,
                                                  pitch=-0.5,
                                                  yaw=-np.pi,
                                                  x=-0.6,
                                                  y=-0.8,
                                                  z=0.0),
            max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi,
                                                  pitch=np.pi,
                                                  yaw=np.pi,
                                                  x=0.8,
                                                  y=0.3,
                                                  z=1.1),
            body_index=station_plant.GetBodyByName("iiwa_link_7", iiwa2_instance).index()))
    
    #builder.Connect(teleop.get_output_port(0),
                    #differential_ik.get_input_port(0))
    
    builder.Connect(teleop.get_output_port(0),
                    differential_ik2.get_input_port(0))

    builder.Connect(station.GetOutputPort("body_poses"),
                    teleop.GetInputPort("body_poses"))
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    #builder.Connect(wsg_teleop.get_output_port(0),
                    #station.GetInputPort("wsg_position"))

    builder.Connect(wsg_teleop.get_output_port(0),
            station.GetInputPort('wsg1_position'))


    # uncomment the following line to see the diagram for the system
    # return diagram 
    '''
    

'\niiwa2_instance = None\nfor i in range(controller_plant2.num_model_instances()):\n    model_instance = ModelInstanceIndex(i)\n    model_instance_name = controller_plant.GetModelInstanceName(model_instance)\n\n    if model_instance_name.startswith(\'iiwa\'):\n        iiwa2_instance = model_instance\n        break\n\ncontroller_plant2 = station.GetSubsystemByName(\n    "iiwa2_controller").get_multibody_plant_for_control()\n\n\nassert(iiwa2_instance != None)\n\n# Set up differential inverse kinematics.\ndifferential_ik2 = AddIiwaDifferentialIK(\n    builder,\n    controller_plant2,\n    frame=controller_plant2.GetFrameByName("iiwa_link_7"))\nbuilder.Connect(differential_ik2.get_output_port(),\n                station.GetInputPort("iiwa2_position"))\nbuilder.Connect(station.GetOutputPort("iiwa2_state_estimated"),\n                differential_ik2.GetInputPort("robot_state"))\n\n# Set up teleop widgets.\n\nteleop = builder.AddSystem(\n    MeshcatPoseSliders(\n        meshcat,\n        min

### Throwing Track Route
Since we have the target position, then if we want to achieve a good throwing task, we need to solve the initial position, initial speed, and initial angle of the iiwa robot.

In [5]:
import time

import numpy as np
from pydrake.all import (AddMultibodyPlantSceneGraph, BsplineTrajectory,
                         DiagramBuilder, KinematicTrajectoryOptimization,
                         MeshcatVisualizer, MeshcatVisualizerParams,
                         MinimumDistanceConstraint, Parser, PositionConstraint,
                         Rgba, RigidTransform, Role, Solve, Sphere,
                         StartMeshcat)

from pydrake.all import (
    MathematicalProgram, Solve, eq, le, ge
)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import PublishPositionTrajectory
from manipulation.scenarios import AddIiwa, AddPlanarIiwa, AddShape, AddWsg
from manipulation.utils import AddPackagePaths, FindResource

In [6]:
# import numpy as np
# true
# import pydrake
# from pydrake.all import (
#     RigidTransform, PiecewiseQuaternionSlerp, RotationMatrix
# )

# from manipulation.meshcat_utils import AddMeshcatTriad

"""
All the function that be used to plann the trajectory
"""



# define the orientation angle in x-y plane to convert 3D problem into 2D problem

def calculate_x_y_orientation(X_A, X_B):
    # the orientation angle that robot will generate trajectory along this direction
    # p_A -> robot base position
    # p_B-> the center of the target position
    p_A = X_A.translation() 
    p_B = X_B.translation() 
    agnle_xy = np.arctan2((p_B[1]-p_A[1]),(p_B[0]-p_A[0]))
    return agnle_xy


# a trajectory for quaternions that are interpolated using piecewise slerp (spherical linear interpolation).
def interpolate_orientation(Rotation_A, Rotation_B):
    # assume scalar t: [0, 1]
    trajectory = PiecewiseQuaternionSlerp()
    # Given a new RotationMatrix, this method adds one segment to the end of this.
    # add the initial rotation matrix A
    trajectory.Append(0.0, Rotation_A)
    # add the final rotation matrix B
    trajectory.Append(1.0, Rotation_B)
    return trajectory


#####
def calculate_rotation_xy_radius(X_A, X_B):
    # X_A = Gripper RigidTransform Pose
    # X_B = robot base RigidTransform Pose
    p_A = X_A.translation() 
    r_A = X_A.rotation()
    p_B = X_B.translation() 
    r_B = X_B.rotation()
    r = np.sqrt((p_B[1]-p_A[1])**2 + (p_B[0]-p_A[0])**2)
    return r

# rotate the robot arm to the orientation direction
# will keep the z axis value, rotate the arm in x-y plane around the z axis
def rotate_to_orientation_direction(X_A, radius, orientation_angle, t):
    p_A = X_A.translation() 
    r_A = X_A.rotation()
    # print(r_A)

    # calculate p_B and r_B
    z_A = p_A[2]   
    x_B = radius*np.cos(orientation_angle)
    y_B = radius*np.sin(orientation_angle)
    z_B = z_A
    p_B = [x_B, y_B, z_B]
    # print(p_B)
    # R = np.array([[1, 0, 0], [0, 1, 0], [0, 0, -1]])
    R1 = RotationMatrix.MakeXRotation(-np.pi/2)
    R_B = R1.multiply(RotationMatrix.MakeYRotation(-orientation_angle))
    # print(R_B)
    X_B = RigidTransform(R_B, p_B)
    p_B = X_B.translation() 
    r_B = X_B.rotation()
    # print(r_B)

    r_interpolate = interpolate_orientation(r_A, r_B)
    r_quaternion = r_interpolate.orientation(t)
    # print(t)
    # print(r_quaternion)
    r_current = RotationMatrix(r_quaternion)
    # print(r_current)

    # calculate the circular curve
    c_radius = radius
    phi_initial = np.arctan2(p_A[1],p_A[0])
    phi_final = orientation_angle
    # theta will change from phi_initial to orientation_angle
    theta = t*(phi_final-phi_initial)

    p_change = np.zeros((3,))
    p_change[0] = c_radius * np.cos(theta+phi_initial)
    p_change[1] = c_radius * np.sin(theta+phi_initial)
    p_change[2] = z_A 
    # print(c_radius)
    # print(p_change)

    p_current = p_change

    X_current = RigidTransform(r_current, p_current)
    return X_current
    
# linear interpolation
def interpolate_pose_linear(X_A, X_B, t):
    # X_A and X_B are RigidTransform Pose
    # X_A is the initial pose
    # X_B is the final pose
    # t:[0, 1] => it can be scaled from [0, T]
    
    p_A = X_A.translation() 
    print(p_A.shape)
    p_B = X_B.translation()
    r_A = X_A.rotation()
    r_B = X_B.rotation()

    r_interpolate = interpolate_orientation(r_A, r_B)
    r_cols = r_interpolate.cols()
    r_rows = r_interpolate.rows()
    
    # The interpolated quaternion at t
    r_quaternion = r_interpolate.orientation(t)
    r_current = RotationMatrix(r_quaternion)
    # print(r_current)

    # position is calculated in a linear line segment
    p_current = p_A + t * (p_B - p_A)
    # print(p_current)

    X_WO_current = RigidTransform(r_current, p_current)
    return X_WO_current


# circular interpolation
def interpolate_pose_circular(X_A, X_B, t):
    # X_A and X_B are RigidTransform Pose
    # X_A is the initial pose
    # X_B is the final pose
    # t:[0, 1] => it can be scaled from [0, T]

    p_A = X_A.translation() 
    p_B = X_B.translation()
    r_A = X_A.rotation()
    r_B = X_B.rotation()

    # interpolate rotation
    # print(r_A)
    # print(r_B)
    r_interpolate = interpolate_orientation(r_A, r_B)
    # print(r_interpolate)

    r_cols = r_interpolate.cols()
    # print(r_cols)
    r_rows = r_interpolate.rows()
    # print(r_rows)
    
    # The interpolated quaternion at t
    r_quaternion = r_interpolate.orientation(t)
    # print(r_quaternion)
    # print(r_quaternion.shape)
    
    r_current = RotationMatrix(r_quaternion)
    # print(r_current)

    # calculate the circular curve
    c_radius = calculate_circular_radius(p_A, p_B)
    phi = calculate_x_y_orientation(X_A, X_B)
    # theta will change from -90 to 0, but we will release the gripper at certain_theta degrees
    # t = 0.5? => release gripper
    theta = (t-1)*np.pi/2

    p_change = np.zeros((3,))
    p_change[0] = c_radius * np.cos(theta) * np.cos(phi)
    p_change[1] = c_radius * np.cos(theta) * np.sin(phi)
    p_change[2] = c_radius + c_radius * np.sin(theta)
    print(c_radius)
    print(p_change)

    p_current = p_A + p_change

    X_WO_current = RigidTransform(r_current, p_current)
    return X_WO_current




In [7]:
def calculate_r_theta_for_throwing(X_A, X_B, gravity):
    # X_A and X_B are RigidTransform Pose
    # X_A is the initial pose after gripper comes to the perparation position for throwing
    # X_B is the target position
    
    p_A = X_A.translation() 
    r_A = X_A.rotation()
    p_B = X_B.translation() 
    r_B = X_B.rotation()
    g = -gravity

    # convert 3d problem to 2d problem
    x_init = np.sqrt((p_A[1])**2 + (p_A[0])**2)
    y_init = p_A[2]

    x_tar = np.sqrt((p_B[1])**2 + (p_B[0])**2)
    y_tar = p_B[2]

    prog = MathematicalProgram()
    

    m = 1 # the mass of the brick

    x = prog.NewContinuousVariables(2)
    # x[0] => r (0.1 - 0.4 m)
    # x[1] => theta (30 - 60 degree)
    r = 0.1 + 0.3*np.random.random()
    # r = 0.3
    theta = np.pi/6 + np.pi/6*np.random.random()
    # theta = np.pi/4
    prog.SetInitialGuess(x,[r,theta])
    
    # cost function: min 0.5*m*v**2 + m*g*(y_0 - y_init) 
    x_0 = x_init + x[0] * np.sin(x[1])
    y_0 = y_init + x[0] - x[0]*np.cos(x[1])
    delta_x = x_tar - x_0
    delta_y = y_tar - y_0
    T = np.sqrt((2/g)*(delta_x*np.tan(x[1])-delta_y))
    v_0 = delta_x/(T*np.cos(x[1]))
    # E = 0.5*m*v_0**2 + m*g*(y_0 - y_init)
    # prog.AddCost(E) 
    def cost_fun(z):
        # cost function: min 0.5*m*v**2 + m*g*(y_0 - y_init) 
        x_0 = x_init + z[0] * np.sin(z[1])
        # print(x_0)
        y_0 = y_init + z[0] - z[0]*np.cos(z[1])
        # print(z[0])
        # print(z[1]*180/np.pi)
        # print(x_0)
        # print(y_0)
        delta_x = x_tar - x_0
        delta_y = y_tar - y_0
        # print(delta_x)
        # print(delta_y)
        T = np.sqrt((2/g)*(delta_x*np.tan(z[1])-delta_y))
        # print(T)
        v_0 = delta_x/(T*np.cos(z[1]))
        print(v_0)
        E = 0.5*m*v_0**2 + m*g*(y_0 - y_init)
        # print(E)

        x_tar_con1 = x_tar - 0.2*np.sqrt(2)
        y_tar_con1 = y_tar + 0.2
        T_con1 = (x_tar_con1 - x_0)/v_0
        # print('  ')
        # print(T_con1)
        # print((T_con1*v_0*np.sin(z[1])-0.5*g*T_con1**2))
        # print((y_tar_con1 - y_0))
        # print((T_con1*v_0*np.sin(z[1])-0.5*g*T_con1**2) - (y_tar_con1 - y_0))
        # print('  ')
        # print(T)
        # print((y_tar - y_0))
        # print((T*v_0*np.sin(z[1])-0.5*g*T**2))
        # print((T*v_0*np.sin(z[1])-0.5*g*T**2) - (y_tar - y_0))
        # print('  ')
        
        return E
    a1 = prog.AddCost(cost_fun, vars=[x[0], x[1]]) 
    # print(a1)
    
    # constraints
    x_tar_con1 = x_tar - 0.2*np.sqrt(2)
    y_tar_con1 = y_tar + 0.2
    T_con1 = (x_tar_con1 - x_0)/v_0


    prog.AddConstraint((T_con1*v_0*np.sin(x[1])-0.5*g*T_con1**2) >= (y_tar_con1 - y_0))
    prog.AddConstraint(x[0]>=0.1)
    prog.AddConstraint(x[0]<=0.4)
    prog.AddConstraint(x[1]>=np.pi/6)
    prog.AddConstraint(x[1]<=np.pi/3)

    result = Solve(prog)

    #   print(result.is_success())
    while result.is_success() == False:
        r = 0.1 + 0.3*np.random.random()
        # r = 0.3
        print(r)
        theta = np.pi/6 + np.pi/6*np.random.random()
        # theta = np.pi/4
        prog.SetInitialGuess(x,[r,theta])
        result = Solve(prog)
    final_result = result.GetSolution()

    # Get the solution
    if (result.is_success()): 
      print("Solution: " + str(result.GetSolution()))

    return final_result 

In [8]:
def calculate_flyingtime_and_initial_speed(X_A, X_B, throwing_r, throwing_angle, gravity):
    # X_A and X_B are RigidTransform Pose
    # X_A is the initial perparation position for throwing
    # X_B is the target position
    
    p_A = X_A.translation() 
    r_A = X_A.rotation()
    p_B = X_B.translation() 
    r_B = X_B.rotation()
    g = -gravity

    # convert 3d problem to 2d problem
    x_init = np.sqrt((p_A[1])**2 + (p_A[0])**2)
    y_init = p_A[2]

    x_tar = np.sqrt((p_B[1])**2 + (p_B[0])**2)
    y_tar = p_B[2]

    m = 1 # the mass of the brick

    r_throw = throwing_r
    angle_throw = throwing_angle

    x_0 = x_init + r_throw * np.sin(angle_throw)
    y_0 = y_init + r_throw - r_throw*np.cos(angle_throw)
    delta_x = x_tar - x_0
    delta_y = y_tar - y_0
    T_fly = np.sqrt((2/g)*(delta_x*np.tan(angle_throw)-delta_y))
    v_initial_throwing = delta_x/(T_fly*np.cos(angle_throw))
    final_result = [T_fly, v_initial_throwing]


    return final_result


def calculate_pose_after_throw_final(X_A, throwing_r):
    # X_A is RigidTransform Pose
    # X_A is the initial perparation position for throwing

    p_A = X_A.translation() 
    r_A = X_A.rotation()

    # convert 3d problem to 2d problem
    x_init = np.sqrt((p_A[1])**2 + (p_A[0])**2)
    angle_orientation_throw = np.arctan2(p_A[1],p_A[0])
    y_init = p_A[2]

    r_throw = throwing_r

    x_final = x_init + r_throw
    y_final = y_init + r_throw

    R_after_throw = r_A.multiply(RotationMatrix.MakeZRotation(-np.pi/2))
    p_after_throw = [x_final*np.cos(angle_orientation_throw), x_final*np.sin(angle_orientation_throw), y_final]
    X_after_throw = RigidTransform(R_after_throw, p_after_throw)
    return X_after_throw


def calculate_pose_after_throw(X_A, throwing_r, throwing_angle):
    # X_A is RigidTransform Pose
    # X_A is the initial perparation position for throwing

    p_A = X_A.translation() 
    r_A = X_A.rotation()

    # convert 3d problem to 2d problem
    x_init = np.sqrt((p_A[1])**2 + (p_A[0])**2)
    angle_orientation_throw = np.arctan2(p_A[1],p_A[0])
    y_init = p_A[2]

    r_throw = throwing_r
    angle_throw = throwing_angle

    x_final = x_init + r_throw*np.sin(angle_throw)
    y_final = y_init + r_throw - r_throw*np.cos(angle_throw)

    R_after_throw = r_A.multiply(RotationMatrix.MakeZRotation(-angle_throw))
    p_after_throw = [x_final*np.cos(angle_orientation_throw), x_final*np.sin(angle_orientation_throw), y_final]
    X_after_throw = RigidTransform(R_after_throw, p_after_throw)
    return X_after_throw



def calculate_speed_base_throw(X_A, throwing_r, throwing_angle):
    # X_A is RigidTransform Pose
    # X_A is the initial perparation position for throwing

    p_A = X_A.translation() 
    r_A = X_A.rotation()

    # convert 3d problem to 2d problem
    x_init = np.sqrt((p_A[1])**2 + (p_A[0])**2)
    y_init = p_A[2]

    r_throw = throwing_r
    angle_throw = throwing_angle

    x_throw = x_init + r_throw * np.sin(angle_throw)
    y_throw = y_init + r_throw - r_throw*np.cos(angle_throw)

    v_x_throw = (-np.pi/2)*r_throw*np.sin(angle_throw)
    v_y_throw = (np.pi/2)*r_throw*np.cos(angle_throw)

    v_throw_base = np.sqrt(v_x_throw**2 + v_y_throw**2)

    return v_throw_base

In [9]:
# we need compute the brick flying trajectory
# given initial throwing pose , initial speed, and flying time, compute the trajectory
def calculate_obejct_flying_trajectory(X_A, X_B, T_fly, initial_speed, initial_angle, gravity):

    # X_A is initial throwing position
    # X_B is the target position

    orientation_angle = calculate_x_y_orientation(X_A, X_B)

    p_A = X_A.translation() 
    r_A = X_A.rotation()
    g = -gravity

    # convert 3d problem to 2d problem
    x_init = np.sqrt((p_A[1])**2 + (p_A[0])**2)
    y_init = p_A[2]

    x_current = x_init + initial_speed * np.cos(initial_angle)*T_fly
    y_current = y_init + initial_speed*np.sin(initial_angle)*T_fly - 0.5*g*T_fly**2

    R_current = r_A
    p_current = [x_current*np.cos(orientation_angle), x_current*np.sin(orientation_angle), y_current]
    X_current = RigidTransform(R_current, p_current)
    
    return X_current


In [10]:
# Test 

###########################################
# Create the pose trajectory
def calc_throwing_traj(initialX_WG, targetX_WG, X_WR=RigidTransform(), t0=0):
    Pose = []

    # convert everything into robot frame
    initialX_WG = X_WR.inverse().multiply(initialX_WG)
    targetX_WG = X_WR.inverse().multiply(targetX_WG)

    X_Gripper_catch_initial = initialX_WG
    X_target = targetX_WG
    p_robot_base = [0,0,0] # We need to calculate this
    r_robot_base = RotationMatrix()
    X_robot_base = RigidTransform(r_robot_base,p_robot_base) # We need to calculate this
    # print("initial pose" + str(initialX_WG))
    # print(targetX_WG)
    # print(X_WR)
    
    angle_oritentation = calculate_x_y_orientation(X_robot_base, X_target)
    print(angle_oritentation*180/np.pi)
    r_rotate = calculate_rotation_xy_radius(X_Gripper_catch_initial, X_robot_base)
    print(r_rotate)


    T = 3.0

    # rotate the arm to the orientation direction
    Pose1 = []
    time_list1 = np.linspace(0, T, 5, endpoint=True)
    print(len(time_list1))
    print(time_list1)
    for i in range(len(time_list1)):
        pose = rotate_to_orientation_direction(X_Gripper_catch_initial, r_rotate, angle_oritentation, time_list1[i]/T)
        Pose1.append(pose)
        print(i)
    # print(Pose1[9])

    # TEST: add to ignore the first 2 poses

    X_after_rotate = Pose1[len(time_list1)-1]
    R_after_rotate = X_after_rotate.rotation()
    p_after_rotate = X_after_rotate.translation() 

    p_throw_initial = [0.4*np.cos(angle_oritentation), 0.4*np.sin(angle_oritentation), 0.2]
    R_throw_initial = R_after_rotate
    X_throw_initial = RigidTransform(R_throw_initial, p_throw_initial)

    Pose2 = []
    time_list2 = np.linspace(0, T, 6, endpoint=True)
    print(len(time_list2))
    print(time_list2)
    for i in range(len(time_list2)):
        pose = interpolate_pose_linear(X_after_rotate, X_throw_initial, time_list2[i]/T)
        Pose2.append(pose)
        print(i)

    # compute throwing radius and throwing angle
    gravity = -9.81
    result1 = calculate_r_theta_for_throwing(X_throw_initial, X_target, gravity)
    throwing_radius = result1[0]
    throwing_theta = result1[1]
    print(result1)
    print(throwing_radius)
    print(throwing_theta*180/np.pi)

    result2 = calculate_flyingtime_and_initial_speed(X_throw_initial, X_target, throwing_radius, throwing_theta, gravity)
    flytime = result2[0]
    v_throw_initial = result2[1]
    print(result2)
    print(flytime)
    print(v_throw_initial)

    X_throw_final = calculate_pose_after_throw_final(X_throw_initial, throwing_radius)
    v_throw_base = calculate_speed_base_throw(X_throw_initial, throwing_radius, throwing_theta)
    print(v_throw_base)
    T_throw_trajectory = v_throw_base/v_throw_initial
    T_throw_trajectory = T_throw_trajectory * 1.2
    print(T_throw_trajectory)
    gripper_release_time = 0 # fix this

    Pose3 = []
    time_list3 = np.linspace(0, T_throw_trajectory, 20, endpoint=True)
    print(len(time_list3))
    print(time_list3)
    for i in range(len(time_list3)):
        pose = interpolate_pose_circular(X_throw_initial, X_throw_final, time_list3[i]/T_throw_trajectory)
        rotation = pose.rotation() @ RotationMatrix.MakeYRotation(np.pi/2)
        Pose3.append(RigidTransform(rotation,pose.translation()))
        print(i)


    

    X_throw = calculate_pose_after_throw(X_throw_initial, throwing_radius, throwing_theta)
    Pose4 = []
    time_list4 = np.linspace(0, flytime, 30, endpoint=True)
    print(len(time_list4))
    print(time_list4)
    for i in range(len(time_list4)):
        pose = calculate_obejct_flying_trajectory(X_throw, X_target, time_list4[i], v_throw_initial, throwing_theta, gravity)
        Pose4.append(pose)
        print(i)
    '''
    for i in range(len(Pose1)):
        X_WStart = Pose1[i]
        meshcat.SetObject('rotate'+str(i)+'good', Sphere(0.01), rgba=Rgba(.9, .1, .1, 1))
        meshcat.SetTransform('rotate'+str(i)+'good', X_WStart)
        AddMeshcatTriad(meshcat,'rotate'+str(i)+'good',length=0.15, radius=0.006,X_PT=X_WStart)

    for i in range(len(Pose2)):
        X_WStart = Pose2[i]
        meshcat.SetObject('linear_to_initial'+str(i)+'good', Sphere(0.01), rgba=Rgba(.9, .1, .1, 1))
        meshcat.SetTransform('linear_to_initial'+str(i)+'good', X_WStart)
        AddMeshcatTriad(meshcat,'linear_to_initial'+str(i)+'good',length=0.15, radius=0.006,X_PT=X_WStart)

    for i in range(len(Pose3)):
        X_WStart = Pose3[i]
        meshcat.SetObject('throwing_tarjectory'+str(i)+'good', Sphere(0.01), rgba=Rgba(.9, .1, .1, 1))
        meshcat.SetTransform('throwing_tarjectory'+str(i)+'good', X_WStart)
        AddMeshcatTriad(meshcat,'throwing_tarjectory'+str(i)+'good',length=0.15, radius=0.006,X_PT=X_WStart)
    '''
    # print('poses before converting')
    # print([Pose1, Pose2, Pose3, Pose4])
    converted_poses = []
    for i, pose in enumerate([Pose1, Pose2, Pose3, Pose4]):
        new_poses = []
        for j in range(len(pose)):
            X_RG = pose[j]
            new_poses.append(X_WR @ X_RG)
        converted_poses.append(new_poses)
    
    # print('poses after converting')
    Pose1, Pose2, Pose3, Pose4 = converted_poses
    # print(converted_poses)

    for i in range(len(Pose4)):
        X_WStart = Pose4[i]
        X_WStart = RigidTransform(X_WStart.rotation(),X_WStart.translation())
        meshcat.SetObject('flying_tarjectory'+str(i)+'good', Sphere(0.01), rgba=Rgba(.9, .1, .1, 1))
        meshcat.SetTransform('flying_tarjectory'+str(i)+'good', X_WStart)
        AddMeshcatTriad(meshcat,'flying_tarjectory'+str(i)+'good',length=0.1, radius=0.006,X_PT=X_WStart)


    meshcat.SetObject("hhh", Sphere(0.02), rgba=Rgba(.9, .1, .1, 1))
    meshcat.SetTransform("hhh", X_target)
    AddMeshcatTriad(meshcat,"hhh",length=0.1, radius=0.006,X_PT=X_target)

    meshcat.SetObject("start1", Sphere(0.02), rgba=Rgba(.9, .1, .1, 1))
    meshcat.SetTransform("start1", X_robot_base)
    AddMeshcatTriad(meshcat,"start1",length=0.1, radius=0.006,X_PT=X_robot_base)

    pose_traj_list = [Pose1,Pose2,Pose3]

    time_list1 = time_list1 + t0
    time_list2 = time_list2 + time_list1[-1] + 0.1
    time_list3 = time_list3 + time_list2[-1] + 0.6

    # delay = 0.0156
    delay = -0.0155

    T_throw = T + T + T_throw_trajectory*throwing_theta/(np.pi/2) + 0.7 + t0 + delay

    pose_traj_times = [time_list1,time_list2,time_list3]

    return pose_traj_list, pose_traj_times, T_throw

### Inverse Kinematics and Dynamics Controller




In [11]:
def GenerateJointAnglesFromPoses(controller_plant, pose_list, wsg_instance_name,d=0.02, angle=1):
    # solve the inverse kinematics problem here
    """Convert end-effector pose list to joint position list using series of
    InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions
    contain gripper joints, but these should not matter to the constraints.
    @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
    @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
    """
    q_knots = []
    world_frame = controller_plant.world_frame()
    gripper_frame = controller_plant.GetFrameByName("body")
    
    # nominal joint for joint-centering.
    wsg_factor = 1 if wsg_instance_name == "wsg1" else 1
    q_nominal = np.array([-1.57, 0.1, 0., -1.2 * wsg_factor, 0., 1.6*wsg_factor,0]) 

    def AddOrientationConstraint(ik, R_WG, bounds):
        """Add orientation constraint to the ik problem. Implements an inequality
        constraint where the axis-angle difference between f_R(q) and R_WG must be
        within bounds. Can be translated to:
        ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
        """
        ik.AddOrientationConstraint(
            frameAbar=world_frame, R_AbarA=R_WG,
            frameBbar=gripper_frame, R_BbarB=RotationMatrix(),
            theta_bound=bounds
        )

    def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
        """Add position constraint to the ik problem. Implements an inequality
        constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
        translated to
        ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
        """
        ik.AddPositionConstraint(
            frameA=world_frame, frameB=gripper_frame, p_BQ=np.zeros(3),
            p_AQ_lower=p_WG_lower, p_AQ_upper=p_WG_upper)

    # print(d)
    for i in range(len(pose_list)):
        
        current_pose = pose_list[i]
        ik = inverse_kinematics.InverseKinematics(controller_plant)
        q_variables = ik.q() # Get variables for MathematicalProgram
        prog = ik.prog() # Get MathematicalProgram

        #### Modify here ###############################

        if i == 0:
            prog.SetInitialGuess(q_variables, q_nominal)
        else:
            prog.SetInitialGuess(q_variables, q_knots[-1])

        diff = q_variables - q_nominal

        prog.AddCost(diff.dot(diff)) # quadratic cost on q_nominal difference

        
        vec = np.array([d/2,d/2,d/2]) # in gripper frame

        AddPositionConstraint(ik, current_pose.translation()-vec, current_pose.translation()+vec)

        angle = 1.
        AddOrientationConstraint(ik, current_pose.rotation(), angle/180. * np.pi)

        ################################################
    
        result = Solve(prog)

        if not result.is_success():
            print(f"IK failure: {current_pose}")
            raise RuntimeError
        solution = result.GetSolution(q_variables)

        # print(f"Solution q:{solution}")
        q_knots.append(solution)
    print(q_knots[0])
    return np.array(q_knots)

In [12]:
# TODO: rename iiwa in the description files
'''
    Adds inverse dynamics controller and connects estimated state input to controller, and output force to the plant
'''
def AddInverseDynamics(builder, cont_plant, iiwa_id = ''):
    kp = [110] * cont_plant.num_positions()
    ki = [1] * cont_plant.num_positions()
    kd = [25] * cont_plant.num_positions()

    id_controller = builder.AddSystem(
        InverseDynamicsController(cont_plant, kp, ki, kd, True))

    # connect estimated robot state to the controller state
    builder.Connect(station.GetOutputPort(f'iiwa{iiwa_id}_state_estimated'),
                id_controller.get_input_port_estimated_state())

    # connect output of inverse dynamics controller to feedforward torque
    builder.Connect(id_controller.get_output_port_control(),
                station.GetInputPort(f'iiwa{iiwa_id}_feedforward_torque'))

    return id_controller

In [13]:
'''
    given the inverse dynamics controller, and the discrete trajectory, creates a continuous trajectory
    and links the positions and feedforward accelerations to the inverse dynamics controller
'''
def CreateTrajectory(t_list, q_knots):
    print("creating trajectory")
    for i in range(len(q_knots)):
        print(f't: {t_list[i]} q: {q_knots[i]}')
    # q_traj = PiecewisePolynomial.CubicWithContinuousSecondDerivatives(t_list, q_knots.T)
    q_traj = PiecewisePolynomial.CubicShapePreserving(t_list, q_knots.T)

    # print("first 0.2 second of trajectory (velocities)")

    # for i in range(20):
    #     ti=t_list[0]+0.2*i/20.
    #     print(f't:{ti} q:{q_traj.derivative().value(ti)}')

    acc_traj = q_traj.derivative(2) # acceleration is the second derivative of the positions

    # generate TrajectorySources for positions and accelerations
    # return TrajectorySource(q_traj, output_derivative_order=1), TrajectorySource(acc_traj)
    return q_traj, acc_traj

def convert_poses_to_robot_frame(poses, X_WR):
    '''
    converts poses in world frame to robot frame

    poses: list of poses X_WP
    X_WR: robot pose in world frame
    '''
    converted_poses = []
    for pose in poses:
        X_WP = pose
        X_RW = X_WR.inverse()
        converted_poses.append(X_RW @ X_WP)

    return converted_poses

### Grasping Objects

Using anti-podal grasping to grasp throwable object



In [14]:
def make_internal_model():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    AddPackagePaths(parser)
    parser.AddAllModelsFromFile(
        FindResource("models/clutter_planning.dmd.yaml"))
    plant.Finalize()
    return builder.Build()

class GraspSelector(LeafSystem):
    def __init__(self, crop_lower, crop_upper, camera_body_indices):
        LeafSystem.__init__(self)
        model_point_cloud = AbstractValue.Make(PointCloud(0))
        
        # initialize input ports
        self.DeclareAbstractInputPort("cloud0", model_point_cloud)
        self.DeclareAbstractInputPort("cloud1", model_point_cloud)
        self.DeclareAbstractInputPort("cloud2", model_point_cloud)
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()]))

        # Initialize output ports
        self.DeclareAbstractOutputPort(
            "X_WG", lambda: AbstractValue.Make(RigidTransform()),
            self.SelectGrasp)


        self._crop_lower = crop_lower
        self._crop_upper = crop_upper
        self._internal_model = make_internal_model()
        self._internal_model_context = self._internal_model.CreateDefaultContext()
        self._rng = np.random.default_rng()
        self._camera_body_indices = camera_body_indices
        


    def SelectGrasp(self, context, output):
        body_poses = self.get_input_port(3).Eval(context)
        pcd = []

        # construct point clouds for each camera
        for i in range(3):
            cloud = self.get_input_port(i).Eval(context)
            pcd.append(cloud.Crop(self._crop_lower, self._crop_upper))
            pcd[i].EstimateNormals(radius=0.1, num_closest=30)
            meshcat.SetObject(f"raw point cloud {self._camera_body_indices[i]}", pcd[i])
            # Flip normals toward camera
            X_WC = body_poses[self._camera_body_indices[i]]
            pcd[i].FlipNormalsTowardPoint(X_WC.translation())
            
        # merge and down sample point clouds
        merged_pcd = Concatenate(pcd)
        meshcat.SetObject("point cloud", merged_pcd)
        down_sampled_pcd = merged_pcd.VoxelizedDownSample(voxel_size=0.005)
        meshcat.SetObject("down sampled point cloud", down_sampled_pcd)
        costs = []
        X_Gs = []

        # Generate antipodal grasp candidate and select candidate with best grasp
        for i in range(100 if running_as_notebook else 2):
            cost, X_G = GenerateAntipodalGraspCandidate(
                self._internal_model, self._internal_model_context,
                down_sampled_pcd, self._rng)
            if np.isfinite(cost):
                costs.append(cost)
                X_Gs.append(X_G)

        if len(costs) == 0:
            # Didn't find a viable grasp candidate
            X_WG = RigidTransform([100,100,100])
            output.set_value(X_WG)
            print("No grasp candidates :(")
            return

        else:
            best = np.argmin(costs)
            X_WG = X_Gs[best]

        AddMeshcatTriad(meshcat, "ideal grasp", length=0.01,
                            radius=0.001, X_PT=X_WG)

        # Output ideal grasp pose
        output.set_value(X_WG)


class PlannerState(Enum):
    GRASPING = 1
    THROWING = 2
    WAIT_FOR_OBJECTS_TO_SETTLE = 3
    WAIT_FOR_OBJECT = 4
    RETURNING = 5
    RESET = 6
    DISABLED = 7

class Planner(LeafSystem):
    def __init__(self, plant, controller_plant,wsg_instance_name):
        LeafSystem.__init__(self)
        self._mode_index = self.DeclareAbstractState(
            AbstractValue.Make(PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE))

        for i in range(plant.num_model_instances()):
            model_instance = ModelInstanceIndex(i)
            model_instance_name = plant.GetModelInstanceName(model_instance)
            if model_instance_name == wsg_instance_name:
                body = plant.GetBodyIndices(model_instance)[0]
        self.wsg_instance_name = wsg_instance_name
        self._gripper_body_index = body
        self._debug = True

        # Initialize input ports
        self.DeclareAbstractInputPort("body_poses", AbstractValue.Make([RigidTransform()]))
        self.DeclareAbstractInputPort("X_WG",AbstractValue.Make(RigidTransform()))

        # Initialize state variables
        self._traj_qG = self.DeclareAbstractState(AbstractValue.Make(PiecewisePolynomial()))
        self._traj_accG = self.DeclareAbstractState(AbstractValue.Make(PiecewisePolynomial()))
        self._traj_wsg_index = self.DeclareAbstractState(AbstractValue.Make(PiecewisePolynomial()))
        self._prev_traj_state = self.DeclareAbstractState(AbstractValue.Make(BasicVector))
        self._gripper_grasp_end_pose = RigidTransform(RollPitchYaw(-np.pi/2, 0, 0),
                                [0.5 ,
                                    0, .55])

        self.throw_target = [x_distance+.5,y_distance,0] if wsg_instance_name == 'wsg' else [0,-0.5,0]
        # if wsg_instance_name != 'wsg':
        #     self.throw_target = RigidTransform(RollPitchYaw(0,0,np.pi/2), [x_distance, y_distance, -0.015]).inverse().multiply(RigidTransform(RotationMatrix(), self.throw_target)).translation()
        self.starting_pose = None
        self._object_detected = None
        self.previous_throw = None

        self._last_joint_angle = None
        # Initialize output ports
        self.DeclareVectorOutputPort("traj_state", 14,self.CalcDesiredState)
        self.DeclareVectorOutputPort('traj_acc', 7,self.CalcDesiredAcc)
        self.DeclareVectorOutputPort("wsg_position", 1, self.CalcWsgPosition)

        # Initialize/declare update events
        self.DeclareInitializationUnrestrictedUpdateEvent(self.Plan)
        self.DeclarePeriodicUnrestrictedUpdateEvent(0.1, 0.0, self.Update)
        self.DeclarePeriodicUnrestrictedUpdateEvent(1, 1.0, self.CheckOperationSpace)
        self.controller_plant = controller_plant
        self.wait_time = 0
        self.timer_lock = False
        self.safe = True

    def CheckOperationSpace(self, context, state):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()
        new_mode = state.get_mutable_abstract_state(int(
            self._mode_index))
        traj_qG = context.get_abstract_state(int(
            self._traj_qG)).get_value()

        if mode == PlannerState.WAIT_FOR_OBJECT and (traj_qG.get_number_of_segments() == 0 or not traj_qG.is_time_in_range(context.get_time())):
            if sum(abs(x) for x in self.get_input_port(1).Eval(context).translation()) > 100 :
                    return
            else:
                new_mode = new_mode.set_value(PlannerState.GRASPING)
        

    def Update(self, context, state):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()
        #print(mode)
        new_mode = state.get_mutable_abstract_state(int(
            self._mode_index))

        traj_qG = context.get_abstract_state(int(
            self._traj_qG)).get_value()

        # Initialize scene objects
        if mode == PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE:
            if context.get_time() > 1.0:
                self.Plan(context, state)
                new_mode = new_mode.set_value(PlannerState.WAIT_FOR_OBJECT)

        # If no trajectories initialized, create one
        if traj_qG.get_number_of_segments() == 0:
            self.Plan(context, state)
            return


        if mode != PlannerState.WAIT_FOR_OBJECT and not traj_qG.is_time_in_range(context.get_time()):
            self.Plan(context, state)



    def Plan(self, context, state):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()
        new_mode = state.get_mutable_abstract_state(int(
            self._mode_index))
        
        if self.starting_pose == None:
            self.starting_pose = self.get_input_port(0).Eval(context)[int(self._gripper_body_index)]

        if mode == PlannerState.THROWING:
            print("Throwing initiated" + self.wsg_instance_name)
            #pose[0] = orient gripper to proper throwing orientation
            #pose[1] = move gripper to initial pose in throwing trajectory
            #pose[2] = throwing trajectory
            initial_pose = self.get_input_port(0).Eval(context)[int(self._gripper_body_index)]

            # if self.wsg_instance_name != 'wsg':
            #     initial_pose = RigidTransform(RollPitchYaw(0,0,np.pi/2), [x_distance, y_distance, -0.015]).inverse().multiply(initial_pose)
            
            pose_traj_list, pose_traj_times, wsg_release_time = calc_throwing_traj(initial_pose,RigidTransform(self.throw_target),X_WR=RigidTransform(RollPitchYaw(0,0,np.pi/2),[x_distance, y_distance, -0.015]) if self.wsg_instance_name != 'wsg' else robot1_transform,t0=context.get_time())
                

            pose_traj_list = np.concatenate(pose_traj_list)

            q_traj_list = []
            
            index = 0
            
            for i in range(len(pose_traj_list)):
                pose = pose_traj_list[i]
                pose = RigidTransform(pose.rotation(),pose.translation()-gripperOffset)
                # if self.wsg_instance_name == 'wsg1':
                #     pose = RigidTransform(RollPitchYaw(0,0,np.pi/2), [x_distance, y_distance, -0.015]).multiply(pose)
                AddMeshcatTriad(meshcat, "throw traj " + str(index), length=0.15, radius=0.006, X_PT=pose_traj_list[i])
                index += 1


            self.previous_throw = pose_traj_list

            if self.wsg_instance_name == "wsg1":
                pose_traj_list = convert_poses_to_robot_frame(pose_traj_list,RigidTransform(RollPitchYaw(0,0,np.pi/2),[x_distance,y_distance,-0.015]))
            else:
                pose_traj_list = convert_poses_to_robot_frame(pose_traj_list,robot1_transform)



            q_traj_list = GenerateJointAnglesFromPoses(self.controller_plant,pose_traj_list,self.wsg_instance_name,d=0.015)
            pose_traj_times = np.concatenate(pose_traj_times)

            print("throw traj times" + str(pose_traj_times));
            q_traj, acc_traj = CreateTrajectory(pose_traj_times, q_traj_list)
            print(f"current time: {context.get_time()}")

            wsg_closed = np.array([0.0])
            wsg_open = np.array([0.107])

            wsg_command = PiecewisePolynomial.FirstOrderHold([context.get_time(),wsg_release_time-0.01],np.hstack([[wsg_closed],
                                                            [wsg_closed]]))

            print("wsg release time: " + str(wsg_release_time))
            wsg_command.AppendFirstOrderSegment(wsg_release_time,wsg_open)

            state.get_mutable_abstract_state(int(
                self._traj_qG)).set_value(q_traj)
            state.get_mutable_abstract_state(int(
                self._traj_accG)).set_value(acc_traj)
            state.get_mutable_abstract_state(int(
                self._traj_wsg_index)).set_value(wsg_command)

            
            new_mode = new_mode.set_value(PlannerState.RETURNING)

        elif mode == PlannerState.RETURNING:
            traj_qG = context.get_abstract_state(int(
            self._traj_qG)).get_value()

            if not traj_qG.is_time_in_range(context.get_time()) and not self.timer_lock:
                self.wait_time = context.get_time()
                print("timer started")
                self.timer_lock = True
                return

            if context.get_time() - self.wait_time > 1.0:
                new_mode = new_mode.set_value(PlannerState.RESET)
                self.timer_lock = False

        elif mode == PlannerState.RESET:
            traj_qG = context.get_abstract_state(int(
            self._traj_qG)).get_value()

            if not traj_qG.is_time_in_range(context.get_time()) and not self.timer_lock:
                self.wait_time = context.get_time()
                print("timer started")
                self.timer_lock = True
                self.safe = False
                return

            if context.get_time() - self.wait_time > 2.0:
                new_mode = new_mode.set_value(PlannerState.WAIT_FOR_OBJECT)
                self.timer_lock = False   

        elif mode == PlannerState.GRASPING:
            '''
            Using ideal grasp candidate pose, X_WG, and current gripper pose,
            generate grasp and escape trajectory
            '''

            if sum(abs(x) for x in self.get_input_port(1).Eval(context).translation()) > 100 :
                return
                
            print("Grasping Planned for " + str(self.wsg_instance_name))
            print([int(self._gripper_body_index)])
            # Specify key trajectory poses for gripper 
            '''
                self.get_input_port(0).Eval(context)
                    [int(self._gripper_body_index)]
            '''
            X_G = {
                "initial":
                    self.get_input_port(0).Eval(context)
                    [int(self._gripper_body_index)],
                "pick":
                    self.get_input_port(1).Eval(context),
                "end":
                    self._gripper_grasp_end_pose if self.wsg_instance_name == "wsg" else
                        RigidTransform(self.get_input_port(0).Eval(context)
                    [int(self._gripper_body_index)].rotation(),self.get_input_port(0).Eval(context)
                    [int(self._gripper_body_index)].translation()+[0,0,-0.18])
                    
            }

            # Generate times and intermediate key poses for gripper
            X_G, times = MakeGripperFrames(X_G,duration=10)
            print(f"Planned {times['postend']} second trajectory ofr {context.get_time()}.")

            # Apply gripper offset for InverseDynamicsController
            for x in X_G:
                R = X_G[x].rotation()
                # X_G[x] = RigidTransform(R,X_G[x].translation())
                X_G[x] = RigidTransform(R,X_G[x].translation()+gripperOffset)


            X_G['prepick'] = X_G['pick']

            if False:  # Useful for debugging
                AddMeshcatTriad(meshcat, "X_Oinitial",length=0.15, radius=0.006, X_PT=X_G["initial"])
                AddMeshcatTriad(meshcat, "X_Gprepare", length=0.15, radius=0.006,X_PT=X_G["prepare"])
                AddMeshcatTriad(meshcat, "X_Gpreend", length=0.15, radius=0.006,X_PT=X_G["preend"])
                AddMeshcatTriad(meshcat, "X_Gplace",length=0.15, radius=0.006, X_PT=X_G["end"])
            
            # Generate complete pose trajectory
            traj_X_G, traj_poses, t_list = MakeGripperPoseTrajectory(X_G, times,steps=50,t0=context.get_time()) # Change to polynomial pose trajectory
            traj_wsg_command = MakeGripperCommandTrajectory(times,t0=context.get_time())

            for pose in range(len(traj_poses)):
                old_X_G = RigidTransform(traj_poses[pose].rotation(),traj_poses[pose].translation()-gripperOffset)
                #old_X_G = RigidTransform(traj_poses[pose].rotation(),traj_poses[pose].translation())
                if self._debug:
                    AddMeshcatTriad(meshcat, "Grab traj " + str(pose),length=0.15, radius=0.006, X_PT=old_X_G)

            if self.wsg_instance_name == "wsg1":
                traj_poses = convert_poses_to_robot_frame(traj_poses,RigidTransform(RollPitchYaw(0,0,np.pi/2),[x_distance,y_distance,-0.015]))
                AddMeshcatTriad(meshcat, "Robot2Pose",length=0.15, radius=0.006, X_PT=RigidTransform(RollPitchYaw(0,0,np.pi/2),[x_distance,y_distance,-0.015]))
            else:
                traj_poses = convert_poses_to_robot_frame(traj_poses,robot1_transform)
               

            # Convert pose trajectory into list of joint space angles using InverseDynamics
            traj_joint_angles = GenerateJointAnglesFromPoses(self.controller_plant,traj_poses,self.wsg_instance_name,d=0.04)


            # Generate joint space and torque trajectory 
            q_traj, acc_traj = CreateTrajectory(t_list, traj_joint_angles)

            # Update state trajectory values to calculated values
            state.get_mutable_abstract_state(int(
                self._traj_qG)).set_value(q_traj)
            state.get_mutable_abstract_state(int(
                self._traj_accG)).set_value(acc_traj)
            state.get_mutable_abstract_state(int(
                self._traj_wsg_index)).set_value(traj_wsg_command)

            new_mode = new_mode.set_value(PlannerState.THROWING)
            self.safe = True

    
    def start_time(self, context):
        return context.get_abstract_state(
            int(self._traj_qG)).get_value().start_time()

    def end_time(self, context):
        return context.get_abstract_state(
            int(self._traj_qG)).get_value().end_time()

    def CalcDesiredState(self, context, output):
        '''
        Output commanded joint angles and joint velocities based on trajectory states
        '''
        mode = context.get_abstract_state(int(self._mode_index)).get_value()
        traj_qG = context.get_abstract_state(int(
            self._traj_qG)).get_value()

        # Command Nothing, when initializing scene
        if not self.safe or mode == PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE or traj_qG.get_number_of_segments() == 0 or (mode == PlannerState.RESET and not traj_qG.is_time_in_range(context.get_time())):
            wsg_factor = 1 if self.wsg_instance_name == "wsg1" else 1
            output.set_value(
                np.concatenate([np.array([-1.57, 0.1, 0., -1.2*wsg_factor, 0., 1.6*wsg_factor, 0.]), np.zeros(7)])
            )
            return

        # Evaluate the trajectory at the current time, and write it to the
        # output port.
 
        if traj_qG.is_time_in_range(context.get_time()):
            q_now = traj_qG.value(context.get_time())
            v_now = traj_qG.derivative().value(context.get_time())
            q_now, v_now = map(np.array, (q_now, v_now))

            output.set_value(
                np.concatenate([q_now, v_now])
            )

            return

        # Command the current position


        q_now = traj_qG.value(self.end_time(context))
        v_now = traj_qG.derivative().value(self.end_time(context))
        q_now, v_now = map(np.array, (q_now, v_now))

        output.set_value(np.concatenate([q_now, v_now]))

    def CalcDesiredAcc(self, context, output):
        '''
        Output commanded joint acceleration based on trajectory states
        '''
        mode = context.get_abstract_state(int(self._mode_index)).get_value()
        traj_accG = context.get_abstract_state(int(
            self._traj_accG)).get_value()



        if mode == PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE or traj_accG.get_number_of_segments() == 0:
            output.set_value(np.zeros(7))
            return

        if traj_accG.is_time_in_range(context.get_time()):
            output.set_value(
                traj_accG.value(context.get_time())
            )
            # output.set_value(
            #     0.0
            # )
            return

        # Command 0 acceleration
        output.set_value(np.zeros(7))

    def CalcWsgPosition(self, context, output):
        '''
        Output commanded gripper poses (open or closed) based on trajectory states
        '''
        opened = np.array([0.107])
        closed = np.array([0.0])
        traj_wsg = context.get_abstract_state(int(
            self._traj_wsg_index)).get_value()
        mode = context.get_abstract_state(int(self._mode_index)).get_value()

        if mode == PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE or traj_wsg.get_number_of_segments() == 0:
            output.SetFromVector([opened])
            return

        # Evaluate the trajectory at the current time, and write it to the
        # output port.
        if traj_wsg.is_time_in_range(context.get_time()):
            output.SetFromVector(traj_wsg.value(context.get_time()))
            return

        # Command the open position
        # output.SetFromVector([opened]) X_RW X_WP X_RP = 

In [15]:
for i in range(station_plant.num_model_instances()):
    model_instance = ModelInstanceIndex(i)
    model_instance_name = station_plant.GetModelInstanceName(model_instance)
    print(model_instance_name)
        

grasp_selector1 = builder.AddSystem(
        GraspSelector(crop_lower=[-.3, -.68, 0.001],
                      crop_upper=[.2, -.39, .3],
                      camera_body_indices=[
                          station_plant.GetBodyIndices(
                              station_plant.GetModelInstanceByName("camera0"))[0],
                          station_plant.GetBodyIndices(
                              station_plant.GetModelInstanceByName("camera1"))[0],
                          station_plant.GetBodyIndices(
                              station_plant.GetModelInstanceByName("camera2"))[0]
                      ]))

builder.Connect(station.GetOutputPort("camera0_point_cloud"),
                grasp_selector1.get_input_port(0))
builder.Connect(station.GetOutputPort("camera1_point_cloud"),
                grasp_selector1.get_input_port(1))
builder.Connect(station.GetOutputPort("camera2_point_cloud"),
                grasp_selector1.get_input_port(2))
builder.Connect(station.GetOutputPort("body_poses"),
                grasp_selector1.GetInputPort("body_poses"))

planner = builder.AddSystem(Planner(station_plant, controller_plant,"wsg"))
builder.Connect(station.GetOutputPort("body_poses"),
                planner.GetInputPort("body_poses"))
builder.Connect(grasp_selector1.GetOutputPort("X_WG"),
                planner.GetInputPort("X_WG"))
builder.Connect(planner.GetOutputPort('wsg_position'),
        station.GetInputPort('wsg_position'))

id_controller1 = AddInverseDynamics(builder, controller_plant)

WorldModelInstance
DefaultModelInstance
iiwa
wsg
iiwa2
wsg1
bin0
bin1
floor
camera0
camera1
camera2
camera3
camera4
camera5
foam_brick
foam_brick2


In [16]:
'''crop_lower=[x_distance-0.12, y_distance-0.28, -1],
                      crop_upper=[x_distance+0.18, y_distance+0.25, .3]'''
grasp_selector2 = builder.AddSystem(
        GraspSelector(crop_lower=[x_distance-0.12+1+-0.5, y_distance-0.26+0.2, 0.001],
                      crop_upper=[x_distance+0.18-0.5+1, y_distance+0.25+0.2, .3],
                      camera_body_indices=[
                          station_plant.GetBodyIndices(
                              station_plant.GetModelInstanceByName("camera3"))[0],
                          station_plant.GetBodyIndices(
                              station_plant.GetModelInstanceByName("camera4"))[0],
                          station_plant.GetBodyIndices(
                              station_plant.GetModelInstanceByName("camera5"))[0]
                      ]))

builder.Connect(station.GetOutputPort("camera3_point_cloud"),
                grasp_selector2.get_input_port(0))
builder.Connect(station.GetOutputPort("camera4_point_cloud"),
                grasp_selector2.get_input_port(1))
builder.Connect(station.GetOutputPort("camera5_point_cloud"),
                grasp_selector2.get_input_port(2))
builder.Connect(station.GetOutputPort("body_poses"),
                grasp_selector2.GetInputPort("body_poses"))

planner2 = builder.AddSystem(Planner(station_plant, controller_plant2, "wsg1"))

builder.Connect(station.GetOutputPort("body_poses"),
                planner2.GetInputPort("body_poses"))
builder.Connect(grasp_selector2.GetOutputPort("X_WG"),
                planner2.GetInputPort("X_WG"))
builder.Connect(planner2.GetOutputPort('wsg_position'),
        station.GetInputPort('wsg1_position'))

id_controller2 = AddInverseDynamics(builder, controller_plant2,'2')

In [17]:
class TrajectorySelector(LeafSystem):
    def __init__(self, num_trajectories):
        LeafSystem.__init__(self)
        
        self._q_trajectories = [AbstractValue.Make(PiecewisePolynomial()) for i in range(num_trajectories)]
        self._acc_trajectories = [AbstractValue.Make(PiecewisePolynomial()) for i in range(num_trajectories)]
        
        for i in range(num_trajectories):
            self.DeclareAbstractInputPort(f'q_trajectory_{i}', self._q_trajectories[i])
            self.DeclareAbstractInputPort(f'acc_trajectory{i}', self._acc_trajectories[i])

        self._traj_index = AbstractValue.Make(int(0))
        self.DeclareAbstractInputPort('trajectory_selector', self._traj_index)

        self.DeclareVectorOutputPort(
            'q_now', 14, self.GetPosition
        )

        self.DeclareVectorOutputPort(
            'acc_now', 7, self.GetAcceleration
        )

    def GetPosition(self, context, output):
        q_trajectories = context.get_abstract_state(
            int(self._q_trajectories)
        ).get_value()

        traj_index = int(context.get_abstract_state(
            int(self._traj_index)
        ).get_value())

        qs = q_trajectories[traj_index].value(context.get_time())
        vs = q_trajectories[traj_index].derivative().value(context.get_time())

        qs, vs = map(np.array, [qs, vs])

        print("before concat")
        output.set_value(
            np.concatenate((qs, vs))
        )
        print("after concat")

        return
    
    def GetAcceleration(self, context, output):
        acc_trajectories = context.get_abstract_state(
            int(self._acc_trajectories)
        ).get_value()

        traj_index = int(context.get_abstract_state(
            int(self._traj_index)
        ).get_value())

        output.set_value(
            acc_trajectories[traj_index].value(context.get_time())
        )

        return

In [18]:
# print("traj_state", planner.GetOutputPort('traj_state'))
# traj_selector = builder.AddSystem(TrajectorySelector(1))

# traj_selector.GetInputPort('trajectory_selector').FixValue(traj_selector.CreateDefaultContext(), 0.)
# builder.Connect(
#     planner.GetOutputPort("traj_state"),
#     traj_selector.GetInputPort('q_trajectory_0')
# )

# builder.Connect(
#     planner.GetOutputPort("traj_acc"),
#     traj_selector.GetInputPort('acc_trajectory_0')
# )

# builder.Connect(traj_selector.GetOutputPort("q_now"),
#                   traj_selector.GetInputPort("desired_state"))
# builder.Connect(traj_selector.GetOutputPort("acc_now"),
#                   id_controller1.GetInputPort("desired_acceleration"))

In [19]:
# print("traj_system", planner.GetOutputPort('traj_source'))

In [20]:
builder.Connect(planner.GetOutputPort("traj_state"),
                   id_controller1.GetInputPort("desired_state"))
builder.Connect(planner.GetOutputPort("traj_acc"),
                   id_controller1.GetInputPort("desired_acceleration"))



builder.Connect(planner2.GetOutputPort("traj_state"),
                   id_controller2.GetInputPort("desired_state"))


builder.Connect(planner2.GetOutputPort("traj_acc"),
                   id_controller2.GetInputPort("desired_acceleration"))

In [21]:
desired_state_logger = pydrake.systems.primitives.LogVectorOutput(planner.GetOutputPort('traj_state'), builder)
estimated_state_logger = pydrake.systems.primitives.LogVectorOutput(station.GetOutputPort('iiwa_state_estimated'), builder)

In [22]:
diagram = builder.Build()
diagram.set_name("project diagram")



In [23]:
simulator = Simulator(diagram)
simulator.AdvanceTo(0.1)
context = simulator.get_mutable_context()



meshcat.Flush()

In [24]:
#SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())

In [25]:
controller_plant2.world_frame().CalcPoseInWorld(context)

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)

In [26]:
if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.Initialize()
        simulator.set_target_realtime_rate(1)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        visualizer.StartRecording(True)
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
            grasp_context = grasp_selector1.GetMyMutableContextFromRoot(context)
        visualizer.PublishRecording()
            
        meshcat.DeleteButton("Stop Simulation")


(3,)
4
(3,)
5
AD{4.2839029096449535, nderiv=2}
AD{4.283903035534594, nderiv=2}
AD{4.198731064176329, nderiv=2}
AD{4.101795410879933, nderiv=2}
AD{4.204472627557207, nderiv=2}
AD{4.07333746558858, nderiv=2}
AD{3.9589641888005924, nderiv=2}
AD{3.942420829714961, nderiv=2}
AD{3.9362525948198037, nderiv=2}
AD{3.9361452018596848, nderiv=2}
AD{3.936144672816661, nderiv=2}
AD{3.936144672816661, nderiv=2}
Solution: [0.4        0.70946477]
[0.4        0.70946477]
0.4
40.64933699024775
[0.6159362586417313, 3.936144672816661]
0.6159362586417313
3.936144672816661
0.6283185307179586
0.19155348685951856
20
[0.         0.01008176 0.02016352 0.03024529 0.04032705 0.05040881
 0.06049057 0.07057234 0.0806541  0.09073586 0.10081762 0.11089939
 0.12098115 0.13106291 0.14114467 0.15122644 0.1613082  0.17138996
 0.18147172 0.19155349]
0.4
[-1.95943488e-17 -1.46957616e-17  0.00000000e+00]
0
0.4
[-0.02642539 -0.01981904  0.0013662 ]
1
0.4
[-0.05267027 -0.0395027   0.00545548]
2
0.4
[-0.07855536 -0.05891652  0

In [27]:
desired_states = desired_state_logger.FindLog(context).data()
estimated_states = estimated_state_logger.FindLog(context).data()

In [28]:
desired_states.shape, estimated_states.shape

((14, 50961), (14, 50961))

In [29]:
error = desired_states-estimated_states
error_sq = np.square(error)
error_sq.mean(axis=1)

array([0.01438062, 0.01691349, 0.01769236, 0.05345158, 0.05696699,
       0.04608922, 0.06845362, 0.11846153, 0.0695222 , 1.05379346,
       0.40220695, 4.53095201, 0.37499993, 1.86251226])

In [30]:
print(desired_states[:, 15000:15010], estimated_states[:, 15000:15010])

[[ 0.30518139  0.30581039  0.3064377   0.30706332  0.30721946  0.30768724
   0.30830945  0.30892995  0.30954873  0.31016578]
 [ 0.26894605  0.26901107  0.26907593  0.26914064  0.2691568   0.2692052
   0.2692696   0.26933385  0.26939794  0.26946187]
 [ 0.7689759   0.76911732  0.76925899  0.76940091  0.76943644  0.7695431
   0.76968554  0.76982824  0.7699712   0.77011442]
 [-1.41723438 -1.41719623 -1.41715816 -1.41712017 -1.41711069 -1.41708226
  -1.41704444 -1.41700669 -1.41696903 -1.41693144]
 [-0.17917583 -0.17924671 -0.1793174  -0.17938789 -0.17940549 -0.1794582
  -0.17952832 -0.17959824 -0.17966797 -0.17973751]
 [ 1.52051035  1.52052554  1.52054075  1.520556    1.52055982  1.52057128
   1.52058658  1.52060191  1.52061727  1.52063267]
 [ 1.66220394  1.66225191  1.66229973  1.66234741  1.6623593   1.66239494
   1.66244234  1.66248959  1.66253669  1.66258366]
 [ 0.62983751  0.62815556  0.62646655  0.62477048  0.62434536  0.62306736
   0.62135718  0.61963995  0.61791566  0.61618431]
 [ 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1e147b87-4611-40c9-8663-f05c09154240' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>